In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder\
.appName("DAG and Broadcast demo")\
.getOrCreate()

In [2]:
orders_data = [
    ("O001","Hyderabad",1200),
    ("O002","Delhi",800),
    ("O003","Mumbai",1500),
    ("O004","Bangalore",400),
    ("O005","Hyderabad",300),
    ("O006","Delhi",2000),
    ("O007","Mumbai",700),
    ("O008","Bangalore",1800),
    ("O009","Delhi",350),
    ("O010","Hyderabad",900)
]

orders_cols = ["order_id","city","order_amount"]

orders_df = spark.createDataFrame(orders_data, orders_cols)
orders_df.show()

+--------+---------+------------+
|order_id|     city|order_amount|
+--------+---------+------------+
|    O001|Hyderabad|        1200|
|    O002|    Delhi|         800|
|    O003|   Mumbai|        1500|
|    O004|Bangalore|         400|
|    O005|Hyderabad|         300|
|    O006|    Delhi|        2000|
|    O007|   Mumbai|         700|
|    O008|Bangalore|        1800|
|    O009|    Delhi|         350|
|    O010|Hyderabad|         900|
+--------+---------+------------+



In [3]:
city_data = [
    ("Hyderabad","Tier-1"),
    ("Delhi","Tier-1"),
    ("Mumbai","Tier-1"),
    ("Bangalore","Tier-1")
]

city_cols = ["city","city_category"]

In [4]:
city_df = spark.createDataFrame(city_data, city_cols)
city_df.show()

+---------+-------------+
|     city|city_category|
+---------+-------------+
|Hyderabad|       Tier-1|
|    Delhi|       Tier-1|
|   Mumbai|       Tier-1|
|Bangalore|       Tier-1|
+---------+-------------+



In [6]:
from pyspark.sql.functions import col

filtered_orders = orders_df.filter(col("order_amount") > 500)

joined_df = filtered_orders.join(
    city_df,
    on="city",
    how="inner"
)

final_df = joined_df.select(
    "order_id",
    "city",
    "city_category",
    "order_amount"
)

In [7]:
final_df.show()

+--------+---------+-------------+------------+
|order_id|     city|city_category|order_amount|
+--------+---------+-------------+------------+
|    O008|Bangalore|       Tier-1|        1800|
|    O002|    Delhi|       Tier-1|         800|
|    O006|    Delhi|       Tier-1|        2000|
|    O001|Hyderabad|       Tier-1|        1200|
|    O010|Hyderabad|       Tier-1|         900|
|    O003|   Mumbai|       Tier-1|        1500|
|    O007|   Mumbai|       Tier-1|         700|
+--------+---------+-------------+------------+



In [8]:
final_df.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=true
+- == Final Plan ==
   ResultQueryStage 2
   +- *(5) Project [order_id#0, city#1, city_category#14, order_amount#2L]
      +- *(5) SortMergeJoin [city#1], [city#13], Inner
         :- *(3) Sort [city#1 ASC NULLS FIRST], false, 0
         :  +- AQEShuffleRead coalesced
         :     +- ShuffleQueryStage 0
         :        +- Exchange hashpartitioning(city#1, 200), ENSURE_REQUIREMENTS, [plan_id=62]
         :           +- *(1) Filter ((isnotnull(order_amount#2L) AND (order_amount#2L > 500)) AND isnotnull(city#1))
         :              +- *(1) Scan ExistingRDD[order_id#0,city#1,order_amount#2L]
         +- *(4) Sort [city#13 ASC NULLS FIRST], false, 0
            +- AQEShuffleRead coalesced
               +- ShuffleQueryStage 1
                  +- Exchange hashpartitioning(city#13, 200), ENSURE_REQUIREMENTS, [plan_id=75]
                     +- *(2) Filter isnotnull(city#13)
                        +- *(2) Scan ExistingRDD[city#1

In [9]:
from pyspark.sql.functions import broadcast
broadcast_join_df=filtered_orders.join(
    broadcast(city_df),
    on="city",
    how="inner"

)

final_broadcast_df=broadcast_join_df.select(
    "order_id",
    "city",
    "city_category",
    "order_amount"
)

In [10]:
final_broadcast_df.show()

+--------+---------+-------------+------------+
|order_id|     city|city_category|order_amount|
+--------+---------+-------------+------------+
|    O001|Hyderabad|       Tier-1|        1200|
|    O002|    Delhi|       Tier-1|         800|
|    O003|   Mumbai|       Tier-1|        1500|
|    O006|    Delhi|       Tier-1|        2000|
|    O007|   Mumbai|       Tier-1|         700|
|    O008|Bangalore|       Tier-1|        1800|
|    O010|Hyderabad|       Tier-1|         900|
+--------+---------+-------------+------------+



In [11]:
final_broadcast_df.explain(True)


== Parsed Logical Plan ==
'Project ['order_id, 'city, 'city_category, 'order_amount]
+- Project [city#1, order_id#0, order_amount#2L, city_category#14]
   +- Join Inner, (city#1 = city#13)
      :- Filter (order_amount#2L > cast(500 as bigint))
      :  +- LogicalRDD [order_id#0, city#1, order_amount#2L], false
      +- ResolvedHint (strategy=broadcast)
         +- LogicalRDD [city#13, city_category#14], false

== Analyzed Logical Plan ==
order_id: string, city: string, city_category: string, order_amount: bigint
Project [order_id#0, city#1, city_category#14, order_amount#2L]
+- Project [city#1, order_id#0, order_amount#2L, city_category#14]
   +- Join Inner, (city#1 = city#13)
      :- Filter (order_amount#2L > cast(500 as bigint))
      :  +- LogicalRDD [order_id#0, city#1, order_amount#2L], false
      +- ResolvedHint (strategy=broadcast)
         +- LogicalRDD [city#13, city_category#14], false

== Optimized Logical Plan ==
Project [order_id#0, city#1, city_category#14, order_amoun

# Broadcast HashJoin


A Broadcast Hash Join is an optimization technique used in Apache Spark to improve the performance of joins when one of the DataFrames being joined is significantly smaller than the other. Instead of shuffling both DataFrames across the network, Spark broadcasts the smaller DataFrame to all nodes in the cluster. This allows each node to perform the join locally without needing to shuffle the larger DataFrame, leading to faster execution times.

In the final_broadcast_df.explain(True) output, you can see BroadcastHashJoin in the Physical Plan. Specifically, the line +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, string, false]),false), [plan_id=344] indicates that the city_df (the smaller DataFrame in this case) was broadcasted to all executors, which then allows the join to be performed more efficiently.